In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import datetime
import sys
from tqdm import tqdm
import os
import random
import pickle
from glob import glob
import gc
from multiprocessing import Pool, cpu_count

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb, inference_xgb
from utils.metric import compute_comptetition_metric
from utils.feature_contena import Features
from utils.pandas_utils import reduce_mem_usage

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = f"""
execution:
    exp_id: exp_135
    debug: False
    submit: False
    multiprocessing: True

dataset: 
    competition_dir: /kaggle/input/child-mind-institute-detect-sleep-states
    cv_split_path: /kaggle/input/cv-split/train_folds.csv
    train_base_path: /kaggle/input/train_base/train_base.csv
    step_csv_dir: /kaggle/input/save_series_csv/csvs

feature:
    agg_freq: 24 # [step]

xgboost:    
    objective: "binary:logistic"
    learning_rate: 0.1  # 0.01で固定。学習時間とのトレードオフ
    reg_alpha: 0.02  # L1正則化。0.1が推奨。
    reg_lambda: 0.2  # L2正則化。0.1が推奨
    random_state: 42
    max_depth: 5  # 3-8。7くらいでいい。
    colsample_bytree: 0.7  # カラムが多い時は少なめ(0.4とか)にする。
    
seed: 46
"""

CFG = yaml.load(config, Loader=yaml.SafeLoader)

CFG["output_dir"] = f"/kaggle/output/{CFG['execution']['exp_id']}"
os.makedirs(CFG["output_dir"], exist_ok=True)

In [4]:
sub = pd.read_csv(os.path.join(CFG["output_dir"], "submission_after_nms.csv"))
sub

,series_id,step,score,event,reduced_score
0,038441c925bb,219395,8.338868,wakeup,8.338868
1,038441c925bb,315875,8.018286,onset,8.018286
2,038441c925bb,333491,7.888499,onset,7.888499
3,038441c925bb,5003,7.579808,onset,7.579808
4,038441c925bb,159995,7.468081,onset,7.468081
...,...,...,...,...,...
248536,fe90110788d2,317195,0.026434,wakeup,0.026434
248537,fe90110788d2,320555,0.026334,onset,0.026334
248538,fe90110788d2,96035,0.026301,wakeup,0.026301
248539,fe90110788d2,149435,0.026171,wakeup,0.026171


In [5]:
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
labels.head()

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [6]:
cv_split = pd.read_csv(CFG['dataset']['cv_split_path'])
sub["fold"] = sub["series_id"].map(cv_split.set_index("series_id")["fold"])
sub = sub[sub["fold"] == 4].reset_index(drop=True)

In [7]:
series_ids = sub["series_id"].unique()
labels_sample = labels[labels["series_id"].isin(series_ids)]
sub_sample = sub[sub["series_id"].isin(series_ids)]
compute_comptetition_metric(labels_sample, sub_sample)[0]

0.777771885252238